In [8]:
pip install pysam 


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
import pysam
import pandas as pd

In [ ]:
import .vcf as Processed_WGS
vcf = pysam.VariantFile(Processed_WGS, "r") # bam = pysam.VariantFile(Processed_WGS, "rb") for a .bam file

In [ ]:
def APOE4_identifier(vcf):
    """
        Checks for the APOE4 mutation seperately from the other mutations due to how highly correlated to AD it is. 

    """
    apoe4_position = 44908684  # APOE4 SNP position on chromosome 19
    apoe4_risk_allele = "C"    # Risk allele associated with Alzheimer's
    apoe4_beta = 1.20177
    apoe4_OR = 3.326

        
    for rec in vcf.fetch("19", apoe4_position - 1, apoe4_position):
        genotype = rec.samples.values()[0]['GT']  # Genotype for the individual
        base_at_apoe4 = rec.ref  # The reference allele at the APOE4 position
        alt_alleles = rec.alts  # List of alternate alleles at this position
        
        if apoe4_risk_allele in alt_alleles:
            if genotype == (0, 1):  # Heterozygous for APOE4 mutation
                print(f"Patient is heterozygous for APOE4 risk allele {apoe4_risk_allele}")
            elif genotype == (1, 1):  # Homozygous for APOE4 risk allele
                print(f"Patient is homozygous for APOE4 risk allele {apoe4_risk_allele}")
            else:
                print("Patient does not have APOE4 risk allele")

    vcf.close()



In [ ]:
gwas_summary = pd.read_excel('cleaned_gwas.xlsx')

In [ ]:
def chrom_loc(gwas_summary, vcf, PRS):
    """
        Finds SNP locations in a genomic sequence, and checks wether they have the risk allele or not.


    """
    PRS = 0
    for mutation_position in gwas_summary['locations']:
        chrom, pos = locations.split(':')
        pos = int(pos)
        effect_size = float(gwas_summary['beta'])

        for rec in vcf.fetch(chrom, pos - 1, pos):
            genotype = rec.samples.values()[0]['GT']  # Genotype for the individual
            base_at_mutation = rec.ref  
            alternative_alleles = rec.alts 

            if gwas_summary['risk Allele'] in alternative_alleles:
                if genotype == (0, 1):
                    PRS += effect_size
                elif genotype == (1, 1):
                    PRS += 2*effect_size
        

        print(PRS)

    vcf.close()
        
    
        

In [12]:
def PRS_calc():
    """
        Calculate PRS and then uses a certain threshold to determine wether teh individual is high risk. 
        Used Clumping and Threshold calulcation method, Otherwise known as PRS(C+T) method.
        Used previous research to determine the threshold of 0.00009735 PRS.

    """